In [1]:
#library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

In [2]:
#mount file from google drive
from google.colab import drive
drive.mount('/content/drive')
#grab the data
img512 = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/img512.npy')
label  = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/label.npy')

Mounted at /content/drive


In [5]:
X512_train = img512
y512_train = label
y512_train = tf.keras.utils.to_categorical(y512_train, 4)

In [6]:
#checkpoint
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [18]:
#resnet
def model_resnet(X_train, y_train,epochs = 8):
    base_model = ap.ResNet50(weights='imagenet',input_shape=(512,512,3),include_top=False)
    base_model.trainable = False

    head_model = base_model.output
    head_model = keras.layers.Flatten()(head_model)
    #head_model = keras.layers.Dense(512, activation = 'relu')(head_model)
    #head_model = keras.layers.Dropout(0.5)(head_model)
    #head_model = keras.layers.Dense(128, activation = 'relu')(head_model)
    head_model = keras.layers.Dense(4,activation='softmax')(head_model)

    model = keras.Model(base_model.input,head_model)
    #model = base_model

    callbacks = [keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/FINAL_DL/logs/resnet'),model_checkpoint_callack]
    
    model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=256, epochs=epochs, callbacks = callbacks, validation_split = 0.2)

    return model,history

In [19]:
model_resnet_m1,history= model_resnet(X512_train,y512_train,12)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/12
10/10 [==============================] - 208s 21s/step - loss: 109.1550 - accuracy: 0.4162 - val_loss: 48.4421 - val_accuracy: 0.4950
Epoch 2/12
10/10 [==============================] - 202s 21s/step - loss: 28.3513 - accuracy: 0.7125 - val_loss: 21.6399 - val_accuracy: 0.7867
Epoch 3/12
10/10 [==============================] - 203s 21s/step - loss: 11.5850 - accuracy: 0.8550 - val_loss: 10.3748 - val_accuracy: 0.8833
Epoch 4/12
10/10 [==============================] - 205s 21s/step - loss: 4.5108 - accuracy: 0.9204 - val_loss: 7.9021 - val_accuracy: 0.9000
Epoch 5/12
10/10 [==============================] - 202s 20s/step - loss: 1.0272 - accuracy: 0.9725 - val_loss: 5.9393 - val_accuracy: 0.9183
Epoch 6/12
10/10 [==============================] - ETA: 0s - loss: 0.3160 - accuracy: 0.9854 INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


10/10 [==============================] - 236s 24s/step - loss: 0.3160 - accuracy: 0.9854 - val_loss: 5.2727 - val_accuracy: 0.9300
Epoch 7/12
10/10 [==============================] - 208s 21s/step - loss: 0.0982 - accuracy: 0.9946 - val_loss: 4.9766 - val_accuracy: 0.9250
Epoch 8/12
10/10 [==============================] - 206s 21s/step - loss: 0.0575 - accuracy: 0.9967 - val_loss: 5.1555 - val_accuracy: 0.9233
Epoch 9/12
10/10 [==============================] - 209s 21s/step - loss: 0.0204 - accuracy: 0.9987 - val_loss: 4.9498 - val_accuracy: 0.9250
Epoch 10/12
10/10 [==============================] - 205s 21s/step - loss: 5.3819e-06 - accuracy: 1.0000 - val_loss: 4.9561 - val_accuracy: 0.9283
Epoch 11/12
10/10 [==============================] - 213s 22s/step - loss: 3.1795e-06 - accuracy: 1.0000 - val_loss: 4.9778 - val_accuracy: 0.9267
Epoch 12/12
10/10 [==============================] - 208s 21s/step - loss: 2.3891e-06 - accuracy: 1.0000 - val_loss: 4.9868 - val_accuracy: 0.9267


In [22]:
model_resnet_m1.save_weights('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/weights/resnet_m1_weights.h5')

In [23]:
#save the curve
his = pd.DataFrame(history.history)
his.to_pickle('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/curve/history_resnet.pkl')

In [24]:
model_resnet_m1.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            